In [1]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
import pymongo
from pymongo import MongoClient

In [2]:
# Connection to mongo
client = MongoClient('mongodb+srv://<user>:<password>@cluster0.l3pqt.mongodb.net/MSA?retryWrites=true&w=majority')
# Select database
db = client['MSA']
# see list of collections
client.MSA.list_collection_names()

['CPI_RegionUrban_raw',
 'LaborForce_clean',
 'employment_prediction',
 'population_clean',
 'GDP_monthly',
 'population_predicted_2024',
 'employment_clean',
 'AvgWeeklyEarningsAllEmp_raw',
 'GDP_raw',
 'unemployment_ROC_2024',
 'employment_raw',
 'OES_raw',
 'msa_codes',
 'unemployment_predicted_2024',
 'arima_unem_pred_score',
 'population_raw',
 'Population_monthly',
 'Employment_clean',
 'LaborForce_raw',
 'Employment_raw',
 'unemployment_raw',
 'GDP_clean',
 'UnemploymentRate_clean',
 'UnemploymentRate_raw',
 'Predicted_2024_ROC_rank_total',
 'Predicted_2024_ROC_rank_total2',
 'GDP_predicted_2024']

In [3]:
# Select the collection with needed data
population = db.population_clean
df = pd.DataFrame(list(population.find()))
print(df.dtypes)
df

_id     object
CBSA     int64
NAME    object
2010     int64
2011     int64
2012     int64
2013     int64
2014     int64
2015     int64
2016     int64
2017     int64
2018     int64
2019     int64
dtype: object


,_id,CBSA,NAME,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,6031813991cc653aeedfe7c7,11540,"Appleton, WI",225948,227548,228656,229978,231627,233146,234200,235811,237011,237974
1,6031813991cc653aeedfe7c9,12020,"Athens-Clarke County, GA",193439,194375,195984,197590,198361,202565,205402,209106,211468,213750
2,6031813991cc653aeedfe7e0,14010,"Bloomington, IL",169810,170909,172721,175037,174105,173141,173070,172754,172410,171517
3,6031813991cc653aeedfe7ed,15540,"Burlington-South Burlington, VT",211538,213235,213962,214827,215676,217152,217447,218975,219939,220411
4,6031813991cc653aeedfe818,19660,"Deltona-Daytona Beach-Ormond Beach, FL",590506,591926,594969,600236,608604,621166,635859,647867,658186,668365
...,...,...,...,...,...,...,...,...,...,...,...,...,...
379,6031813991cc653aeedfe8ca,38340,"Pittsfield, MA",131317,130547,130284,129488,128856,127858,127067,126353,125901,124944
380,6031813991cc653aeedfe8ef,41700,"San Antonio-New Braunfels, TX",2153009,2193923,2236981,2280002,2328894,2379232,2426276,2472121,2512379,2550960
381,6031813991cc653aeedfe8f3,42100,"Santa Cruz-Watsonville, CA",263147,264858,266340,268816,270861,273503,274396,274856,273841,273213
382,6031813991cc653aeedfe908,44180,"Springfield, MO",437349,440432,444329,448427,451556,455593,457917,462090,466273,470300


In [4]:
df.rename(columns={'NAME':'MSA'}, inplace=True)
df.head()

,_id,CBSA,MSA,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,6031813991cc653aeedfe7c7,11540,"Appleton, WI",225948,227548,228656,229978,231627,233146,234200,235811,237011,237974
1,6031813991cc653aeedfe7c9,12020,"Athens-Clarke County, GA",193439,194375,195984,197590,198361,202565,205402,209106,211468,213750
2,6031813991cc653aeedfe7e0,14010,"Bloomington, IL",169810,170909,172721,175037,174105,173141,173070,172754,172410,171517
3,6031813991cc653aeedfe7ed,15540,"Burlington-South Burlington, VT",211538,213235,213962,214827,215676,217152,217447,218975,219939,220411
4,6031813991cc653aeedfe818,19660,"Deltona-Daytona Beach-Ormond Beach, FL",590506,591926,594969,600236,608604,621166,635859,647867,658186,668365


In [5]:
df.dtypes

_id     object
CBSA     int64
MSA     object
2010     int64
2011     int64
2012     int64
2013     int64
2014     int64
2015     int64
2016     int64
2017     int64
2018     int64
2019     int64
dtype: object

In [6]:
df.drop(columns='_id', inplace=True)
df.head()

,CBSA,MSA,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,11540,"Appleton, WI",225948,227548,228656,229978,231627,233146,234200,235811,237011,237974
1,12020,"Athens-Clarke County, GA",193439,194375,195984,197590,198361,202565,205402,209106,211468,213750
2,14010,"Bloomington, IL",169810,170909,172721,175037,174105,173141,173070,172754,172410,171517
3,15540,"Burlington-South Burlington, VT",211538,213235,213962,214827,215676,217152,217447,218975,219939,220411
4,19660,"Deltona-Daytona Beach-Ormond Beach, FL",590506,591926,594969,600236,608604,621166,635859,647867,658186,668365


In [7]:
# for loop to predict 2024 values
predictions = []
for i in range(0,384):
    y = df.iloc[i, 2:].values
    series = pd.Series(y, dtype='int')
    model = ARIMA(series, order=(2, 1, 1))
    model_fit = model.fit()
    pred = model_fit.forecast(5)
    predictions.append(pred.values.tolist()[-1])
df['2024'] = predictions

C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: Convergen

C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\J

C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\J

C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\J

In [8]:
df

,CBSA,MSA,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2024
0,11540,"Appleton, WI",225948,227548,228656,229978,231627,233146,234200,235811,237011,237974,2.437468e+05
1,12020,"Athens-Clarke County, GA",193439,194375,195984,197590,198361,202565,205402,209106,211468,213750,2.257594e+05
2,14010,"Bloomington, IL",169810,170909,172721,175037,174105,173141,173070,172754,172410,171517,1.714382e+05
3,15540,"Burlington-South Burlington, VT",211538,213235,213962,214827,215676,217152,217447,218975,219939,220411,2.246917e+05
4,19660,"Deltona-Daytona Beach-Ormond Beach, FL",590506,591926,594969,600236,608604,621166,635859,647867,658186,668365,-2.291451e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...
379,38340,"Pittsfield, MA",131317,130547,130284,129488,128856,127858,127067,126353,125901,124944,1.249616e+05
380,41700,"San Antonio-New Braunfels, TX",2153009,2193923,2236981,2280002,2328894,2379232,2426276,2472121,2512379,2550960,2.754410e+06
381,42100,"Santa Cruz-Watsonville, CA",263147,264858,266340,268816,270861,273503,274396,274856,273841,273213,2.739410e+05
382,44180,"Springfield, MO",437349,440432,444329,448427,451556,455593,457917,462090,466273,470300,4.885766e+05


In [9]:
df.sort_values(by='2024').head(10)

,CBSA,MSA,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2024
4,19660,"Deltona-Daytona Beach-Ormond Beach, FL",590506,591926,594969,600236,608604,621166,635859,647867,658186,668365,-229145.141871
313,16180,"Carson City, NV",54981,54676,54312,53726,54082,54068,54147,54532,55202,55916,55774.336234
232,47460,"Walla Walla, WA",58915,59444,59329,59361,59510,59932,60008,60512,60615,60760,60781.999741
31,21420,"Enid, OK",60765,60763,61399,62370,62790,63248,62493,61600,61095,61056,60934.264877
200,30300,"Lewiston, ID-WA",61040,61409,61413,61927,62031,62108,62422,62798,62872,62990,63984.716808
145,19180,"Danville, IL",81640,81414,80862,80585,79783,79170,78494,77689,76702,75758,74818.246110
7,24260,"Grand Island, NE",72936,73547,74238,74652,75240,75319,75598,75456,75476,75553,75800.452504
211,16220,"Casper, WY",75470,76399,78565,81103,81377,82137,80909,79586,79175,79858,79816.886179
166,25980,"Hinesville, GA",77339,80267,80406,78825,81120,79269,79845,80399,79697,80994,81070.512643
9,24500,"Great Falls, MT",81507,81714,81622,82194,82056,81878,81562,81633,81688,81366,81350.506251


In [10]:
df.drop([4], inplace=True)
df

,CBSA,MSA,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2024
0,11540,"Appleton, WI",225948,227548,228656,229978,231627,233146,234200,235811,237011,237974,2.437468e+05
1,12020,"Athens-Clarke County, GA",193439,194375,195984,197590,198361,202565,205402,209106,211468,213750,2.257594e+05
2,14010,"Bloomington, IL",169810,170909,172721,175037,174105,173141,173070,172754,172410,171517,1.714382e+05
3,15540,"Burlington-South Burlington, VT",211538,213235,213962,214827,215676,217152,217447,218975,219939,220411,2.246917e+05
5,20100,"Dover, DE",162956,165110,167297,169038,171456,173128,174542,176499,178540,180786,1.904098e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...
379,38340,"Pittsfield, MA",131317,130547,130284,129488,128856,127858,127067,126353,125901,124944,1.249616e+05
380,41700,"San Antonio-New Braunfels, TX",2153009,2193923,2236981,2280002,2328894,2379232,2426276,2472121,2512379,2550960,2.754410e+06
381,42100,"Santa Cruz-Watsonville, CA",263147,264858,266340,268816,270861,273503,274396,274856,273841,273213,2.739410e+05
382,44180,"Springfield, MO",437349,440432,444329,448427,451556,455593,457917,462090,466273,470300,4.885766e+05


In [11]:
df['2024'] = df['2024'].astype('int')
df

,CBSA,MSA,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2024
0,11540,"Appleton, WI",225948,227548,228656,229978,231627,233146,234200,235811,237011,237974,243746
1,12020,"Athens-Clarke County, GA",193439,194375,195984,197590,198361,202565,205402,209106,211468,213750,225759
2,14010,"Bloomington, IL",169810,170909,172721,175037,174105,173141,173070,172754,172410,171517,171438
3,15540,"Burlington-South Burlington, VT",211538,213235,213962,214827,215676,217152,217447,218975,219939,220411,224691
5,20100,"Dover, DE",162956,165110,167297,169038,171456,173128,174542,176499,178540,180786,190409
...,...,...,...,...,...,...,...,...,...,...,...,...,...
379,38340,"Pittsfield, MA",131317,130547,130284,129488,128856,127858,127067,126353,125901,124944,124961
380,41700,"San Antonio-New Braunfels, TX",2153009,2193923,2236981,2280002,2328894,2379232,2426276,2472121,2512379,2550960,2754410
381,42100,"Santa Cruz-Watsonville, CA",263147,264858,266340,268816,270861,273503,274396,274856,273841,273213,273940
382,44180,"Springfield, MO",437349,440432,444329,448427,451556,455593,457917,462090,466273,470300,488576


In [12]:
df['2024_Pop_ROC'] = (df['2024']-df['2019'])/df['2019'] *100
df

,CBSA,MSA,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2024,2024_Pop_ROC
0,11540,"Appleton, WI",225948,227548,228656,229978,231627,233146,234200,235811,237011,237974,243746,2.425475
1,12020,"Athens-Clarke County, GA",193439,194375,195984,197590,198361,202565,205402,209106,211468,213750,225759,5.618246
2,14010,"Bloomington, IL",169810,170909,172721,175037,174105,173141,173070,172754,172410,171517,171438,-0.046060
3,15540,"Burlington-South Burlington, VT",211538,213235,213962,214827,215676,217152,217447,218975,219939,220411,224691,1.941827
5,20100,"Dover, DE",162956,165110,167297,169038,171456,173128,174542,176499,178540,180786,190409,5.322868
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379,38340,"Pittsfield, MA",131317,130547,130284,129488,128856,127858,127067,126353,125901,124944,124961,0.013606
380,41700,"San Antonio-New Braunfels, TX",2153009,2193923,2236981,2280002,2328894,2379232,2426276,2472121,2512379,2550960,2754410,7.975429
381,42100,"Santa Cruz-Watsonville, CA",263147,264858,266340,268816,270861,273503,274396,274856,273841,273213,273940,0.266093
382,44180,"Springfield, MO",437349,440432,444329,448427,451556,455593,457917,462090,466273,470300,488576,3.886030


In [13]:
df_pred = df[['CBSA','MSA', '2024_Pop_ROC']].copy()
df_pred

,CBSA,MSA,2024_Pop_ROC
0,11540,"Appleton, WI",2.425475
1,12020,"Athens-Clarke County, GA",5.618246
2,14010,"Bloomington, IL",-0.046060
3,15540,"Burlington-South Burlington, VT",1.941827
5,20100,"Dover, DE",5.322868
...,...,...,...
379,38340,"Pittsfield, MA",0.013606
380,41700,"San Antonio-New Braunfels, TX",7.975429
381,42100,"Santa Cruz-Watsonville, CA",0.266093
382,44180,"Springfield, MO",3.886030


In [14]:
df_pred['Pop_Score'] = df_pred['2024_Pop_ROC'].rank(ascending=True)

In [15]:
df_pred

,CBSA,MSA,2024_Pop_ROC,Pop_Score
0,11540,"Appleton, WI",2.425475,236.0
1,12020,"Athens-Clarke County, GA",5.618246,323.0
2,14010,"Bloomington, IL",-0.046060,52.0
3,15540,"Burlington-South Burlington, VT",1.941827,218.0
5,20100,"Dover, DE",5.322868,313.0
...,...,...,...,...
379,38340,"Pittsfield, MA",0.013606,117.0
380,41700,"San Antonio-New Braunfels, TX",7.975429,354.0
381,42100,"Santa Cruz-Watsonville, CA",0.266093,165.0
382,44180,"Springfield, MO",3.886030,280.0


In [16]:
# create new collection in mongo
arima_pop_pred_score = db.arima_pop_pred_score

In [17]:
df_dict = df_pred.to_dict(orient='records')

In [18]:
arima_pop_pred_score.insert_many(df_dict)